In [46]:
# 找出在cake人力銀行中，搜尋「資料工程師」的職缺資料

import urllib.request as req
import bs4 as bs

import pandas as pd

import time

# 嘗試處理，抓到後面沒資料的話，出現http 404，設定第二組try...except
import requests
from urllib.error import HTTPError

# 準備一個空白的list
table = [] # Clear the table list before scraping

# 抓100頁資料，如遇問題可繼續，並會顯示錯誤為何（比如IndexError或是too many requests）
for i in range(100):
    page = i + 1
    # Corrected URL to include page number，需要原本的網址後面加上?page=
    url = "https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=" + str(page)
    print("頁數:", url)
    # 添加了對request清單長度的檢查，以防止任何職位清單未包含所有預期元素時出現潛在的 IndexError 錯誤。
    # 添加了 try...except 程式碼區塊，用於捕獲抓取過程中的任何其他潛在錯誤，並允許程式碼繼續處理後續頁面。

    time.sleep(3)
    # 使用 try，測試內容是否正確
    try:
        resp = req.urlopen(url)
        content = resp.read()
        html = bs.BeautifulSoup(content)

        # find和find_all是區塊的專屬功能，抓取一個div（區塊）有哪些符合搜尋條件的，而其產出的狀態為 list。
        # js為jobs（多個），j為job（單個）
        js = html.find_all("div", {"class":"JobSearchItem_container__oKoBL"})
        # 以上find_all萃取出 所有職缺區塊
        for j in js:
            # 欲抓取的資料
            t = j.find("a", {"class":"JobSearchItem_jobTitle__bu6yO"})# t為職缺名稱
            c = j.find("a", {"class":"JobSearchItem_companyName__bY7JI"})# c為公司名稱
            request = j.find_all("div", {"class":"InlineMessage_label__LJGjW"})# 表需要的員工條件，有多個資料共用同一個區塊

            # 在訪問所需元素之前，檢查它們是否存在
            # 此部分為解決依課程方式抓m會出現「IndexError: list index out of range」
            # 以下if...至else為colab的gemini修改產生，何時要設text何時不用設，該邏輯尚須研究
            # 邏輯初步猜測：薪水和工作經驗是敘述性，需求人數、工作地點、管理責任是標籤？
            if len(request) > 4:
                jo = request[0]# jo為需求人數，job opening
                l = request[1]# l為工作地點
                s_text = request[2].get_text().strip()# s為薪資範圍
                ex_text = request[3].get_text().strip()# ex為工作年資
                m = request[4]# m為管理責任
            else:
                # 處理某些元素缺失的情況
                jo = None
                l = None
                s_text = None
                ex_text = None
                m = None

            # m = resquest[4]# m為管理責任，此階段出現「IndexError: list index out of range」
            jd = j.find("div",{"class":"JobSearchItem_description__si5zg"})# Jd為工作說明



            # 欲產生的資料
            # 以下if和else為chatgpt產生，該邏輯尚須研究
            t_text = t.get_text().strip() if t else None
            c_text = c.get_text().strip() if c else None
            jo_text = jo.get_text().strip() if jo else None
            l_text = l.get_text().strip() if l else None
            # s = s.get_text().strip() 原本設定要做，但系統直接將text設定在上面，可忽略
            # ex = ex.get_text().strip() 原本設定要做，但系統直接將text設定在上面，可忽略
            m = m.get_text().strip() if m else None
            jd_text = jd.get_text().strip() if jd else None
            t_href = "www.cake.me/" + t["href"] if t and "href" in t.attrs else None

            # 一筆資料就是一個字典
            row = {
                "職務名稱:":t_text,
                "公司名稱:":c_text,
                "需求人數:":jo_text,
                "工作地點:":l_text,
                "薪資範圍:":s_text,
                "工作年資:":ex_text,
                "管理責任:":m,
                "工作說明:":jd_text,
                "網站連結:":t_href,
            }
            table.append(row)

            # 呈現的資料（前測檢視用）
            # print("職務名稱:",t_text)
            # print("公司名稱:",c_text)
            # print("需求人數:",jo_text)
            # print("工作地點:",l_text)
            # print("薪資範圍:",s)
            # print("工作年資:",ex)
            # print("管理責任:",m)
            # print("工作說明:",jd_text)
            # print("網站連結:",t_href)
            # 分隔線
            # print("-" * 30)
    # 如果 try 的內容發生錯誤，就執行 except 裡的內容
    except Exception as e:
        print(f"第 {page} 頁發生錯誤：{e}")
        # 如果出現錯誤，還是繼續下一頁
        # 使用 f-string 將變數 page 插入字串並印出，讓使用者知道第幾頁出現什麼錯誤
        # 該指令等同「print("第" + str(page) + "頁發生錯誤：" + str(e))」


# 進一步顯示是否有成功抓取或失敗，抓到後面頁數沒資料的話，便會停止爬蟲
    # 以下為chatgpt提供之作法，嘗試成功
    # 使用 try，測試內容是否正確
    try:
        response = requests.get(url) # 套用requests第三方套件，帶入欲查詢的網址
        response.raise_for_status()  # 若不是HTTP 200，會觸發HTTPError（觸發Exception）。
        data = response.text # 取得回應內容並以字串型態存到data。 requests會試圖自動偵測編碼並進行解碼。

        # 若頁面有明顯「無資料(頁面文字包含no jobs found或字元數小於 500 → 視為空頁）」則停止爬取
        if "No jobs found" in data or len(data) < 500:
            print(f"第 {page} 頁沒有資料，停止爬取。")
            # 使用 f-string 將變數 page 插入字串並印出，讓使用者知道第幾頁無資料
            # 該指令等同「print("第" + str(page) + "頁沒有資料，停止爬取。")」
            break
            # 沒有資料便break

        print(f"成功抓取第 {page} 頁")
        # 使用 f-string 將變數 page 插入字串，有資料且成功抓取就會印出
        # 該指令等同「print("成功抓取第" + str(page) + "頁")」

    # 如果 try 的內容發生錯誤，就執行 except 裡的內容
    # 捕捉 response.raise_for_status() 所拋出的 HTTPError
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404: # 伺服器給你的回應狀態（整數，例如 200、404）
        # 若狀態碼為404（頁面不存在），表示可能已到最後一頁、或分頁號碼超過範圍
            print(f"第 {page} 頁不存在（404），停止爬取。")
            # 使用 f-string 將變數 page 插入字串，讓使用者知道第幾頁不存在（404）
            # 該指令等同「print("第" + str(page) + "頁不存在（404），停止爬取。")」
            break
        else:
            continue

# 轉換存檔
df = pd.DataFrame(table)
df.to_csv("cakejob.csv", encoding="utf-8")
df

頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=1
成功抓取第 1 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=2
成功抓取第 2 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=3
成功抓取第 3 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=4
成功抓取第 4 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=5
成功抓取第 5 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=6
成功抓取第 6 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=7
成功抓取第 7 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=8
成功抓取第 8 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=9
成功抓取第 9 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=10
成功抓取第 10 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?pag

,職務名稱:,公司名稱:,需求人數:,工作地點:,薪資範圍:,工作年資:,管理責任:,工作說明:,網站連結:
0,IT Data Engineer,TSMC 台積電,1・Full-time・Entry level,"Hsinchu, North District, Hsinchu City, Taiwan・...",40K+ TWD / month,No requirement for relevant working experience,No management responsibility,Established in 1987 and headquartered in Taiwa...,www.cake.me//companies/tsmc/jobs/data-engineer...
1,數據工程師實習生 Data Engineer Intern,COMMEET＿擁樂數據服務股份有限公司,1・Internship・Entry level,"Songshan District, Taipei City, Taiwan 105・Tai...",200 ~ 250 TWD / hour,No requirement for relevant working experience,No management responsibility,COMMEET (https://commeet.co) 擴大徵才，歡迎有熱情之新夥伴加入我...,www.cake.me//companies/commeet/jobs/data-assis...
2,Data Engineer,Pinkoi,None,None,None,None,None,【關於 Pinkoi Data Squad】我們是 Pinkoi 的資料團隊，其目標為打造有...,www.cake.me//companies/pinkoi/jobs/b2ec807e-05...
3,資料工程師,上洋股份有限公司,1・Full-time・Assistant,"Wugu District, New Taipei, Taiwan",50K+ TWD / month,2 years of experience required,No management responsibility,數據驅動決策與營運最佳化是上洋邁向智慧營運的重要一步。我們正在尋找一位對資料工程具備熱情與專...,www.cake.me//companies/upyoung/jobs/data-engin...
4,Data Engineer,WorldQuant,None,None,None,None,None,WorldQuant develops and deploys systematic fin...,www.cake.me//companies/WorldQuant/jobs/4607049...
...,...,...,...,...,...,...,...,...,...
519,Saker Squad | 軟體 × Embedded 雙棲開發,神準科技股份有限公司,5・Full-time・Entry level,"Nangang District, Taipei City, Taiwan・Guishan ...",80K ~ 90K TWD / month,No requirement for relevant working experience,No management responsibility,打造下一支橫跨韌體、雲端、應用層的軟體獵隼小隊！\n🚀 計畫簡介\nSaker Squad專...,www.cake.me//companies/senaonetworks/jobs/sake...
520,Junior .NET Core Developer,創科數碼有限公司,1・Full-time・Entry level,Taiwan,55K ~ 70K TWD / month,2 years of experience required,No management responsibility,【職務內容】\n我們正在尋找對後端開發有熱情的 .NET 工程師，加入我們的產品開發團隊。你...,www.cake.me//companies/linkedin-company-992175...
521,產品助理實習生（產品規格設計與開發協作）,零一成長行銷有限公司,1・Internship・Internship,"Taipei City, Taiwan",190 ~ 250 TWD / hour,No requirement for relevant working experience,No management responsibility,公司/團隊簡介（About Us）\n零一行銷（01 Marketing） 是 SEO 實驗...,www.cake.me//companies/growthmarketing/jobs/pr...
522,🍰【女生聲優實習生 Voice Actor Intern】— 加入我們，讓聲音為品牌說話！,深智科技股份有限公司,1・Internship・Internship,"New Taipei, Taiwan",250 ~ 500 TWD / hour,No requirement for relevant working experience,No management responsibility,🍰【女生聲優實習生 Voice Actor Intern】— 加入我們，讓聲音為品牌說話！\...,www.cake.me//companies/nexretail/jobs/296a9b


In [ ]:
job_title_distribution = df['職務名稱:'].value_counts()
display(job_title_distribution)

,count
職務名稱:,
Data Engineer,4
後端工程師,3
前端工程師,2
Senior - Software Engineer,2
"Software Engineering Manager II, Google Cloud",2
...,...
"Engineering Manager, Core Data Processing",1
"Senior Engineering Manager, Data Discovery",1
"Firmware Engineer, Pixel Power",1


In [ ]:
df = pd.DataFrame(table)
display(df.head())

""


In [ ]:
pip install selenium beautifulsoup4 pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 32.5 MB/s eta 0:00:00


In [70]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

BASE_URL = "https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB"
PARAMS = {
    "professions[0]": "it_data-engineer",
    "page": 1
}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

def fetch_page(page):
    """抓取單一頁面 HTML（含錯誤處理）"""
    PARAMS["page"] = page
    try:
        resp = requests.get(BASE_URL, params=PARAMS, headers=headers, timeout=10)
        resp.raise_for_status()
        return resp.text
    except requests.exceptions.HTTPError as e:
        if resp.status_code == 404:
            print(f"❌ 第 {page} 頁不存在（404）。")
            return None
        print(f"⚠️ HTTP 錯誤：{e}")
        return None
    except Exception as e:
        print(f"⚠️ 其他錯誤：{e}")
        return None


def parse_job_list(html):
    """解析單頁職缺列表"""
    soup = BeautifulSoup(html, "html.parser")
    jobs = soup.find_all("div", class_="JobSearchItem_container__oKoBL")
    return jobs


def extract_job_info(job):
    """從單筆職缺區塊提取資料"""

    # 職缺名稱 + 網址
    title_tag = job.find("a", class_="JobSearchItem_jobTitle__bu6yO")
    job_title = title_tag.get_text(strip=True) if title_tag else None
    job_url = "https://www.cake.me" + title_tag["href"] if title_tag and "href" in title_tag.attrs else None

    # 公司名稱
    company_tag = job.find("a", class_="JobSearchItem_companyName__bY7JI")
    company_name = company_tag.get_text(strip=True) if company_tag else None

    # Inline 標籤（需求人數、地點、薪資、經驗、管理責任）
    tags = job.find_all("div", class_="InlineMessage_label__LJGjW")

    hire_info = tags[0].get_text(strip=True) if len(tags) > 0 else None  # 需求人數（含職務類別/等級）
    location = tags[1].get_text(strip=True) if len(tags) > 1 else None
    salary = tags[2].get_text(strip=True) if len(tags) > 2 else None
    experience = tags[3].get_text(strip=True) if len(tags) > 3 else None
    management = tags[4].get_text(strip=True) if len(tags) > 4 else None

    # 工作內容
    desc_tag = job.find("div", class_="JobSearchItem_description__si5zg")
    job_desc = desc_tag.get_text(strip=True) if desc_tag else None

    # 拆解 hire_info → 招募人數 / 職務類別 / 職務等級
    num, job_type, job_level = None, None, None
    if hire_info and "・" in hire_info:
        parts = hire_info.split("・")
        if len(parts) >= 1: num = parts[0]
        if len(parts) >= 2: job_type = parts[1]
        if len(parts) >= 3: job_level = parts[2]

    return {
        "職務名稱": job_title,
        "公司名稱": company_name,
        "招募人數": num,
        "職務類別": job_type,
        "職務等級": job_level,
        "工作地點": location,
        "薪資範圍": salary,
        "工作年資": experience,
        "管理責任": management,
        "工作說明": job_desc,
        "網站連結": job_url
    }


# -------------------------
# 主程式（開始爬）
# -------------------------

all_jobs = []

for page in range(1, 100):  # 至多爬 100 頁
    print(f"📄 正在抓取第 {page} 頁...")

    html = fetch_page(page)
    if not html:
        print("❌ 停止爬取（HTML None 或 404）")
        break

    jobs = parse_job_list(html)

    # 當沒有職缺 → 停止
    if len(jobs) == 0:
        print("⭕ 無更多資料，結束")
        break

    for job in jobs:
        info = extract_job_info(job)
        all_jobs.append(info)

    print(f"✔ 成功抓取 {len(jobs)} 筆")

    time.sleep(2)  # 避免被封鎖


# -------------------------
# 存成 CSV
# -------------------------

df = pd.DataFrame(all_jobs)
df.to_csv("cake_job_clean.csv", encoding="utf-8", index=False)

print("🎉 完成！資料共", len(df), "筆")
df.head()


📄 正在抓取第 1 頁...
✔ 成功抓取 10 筆
📄 正在抓取第 2 頁...
✔ 成功抓取 10 筆
📄 正在抓取第 3 頁...
✔ 成功抓取 10 筆
📄 正在抓取第 4 頁...
✔ 成功抓取 7 筆
📄 正在抓取第 5 頁...
❌ 第 5 頁不存在（404）。
❌ 停止爬取（HTML None 或 404）
🎉 完成！資料共 37 筆


,職務名稱,公司名稱,招募人數,職務類別,職務等級,工作地點,薪資範圍,工作年資,管理責任,工作說明,網站連結
0,ITDataEngineer,TSMC 台積電,1,Full-time,Entry level,"Hsinchu, North District, Hsinchu City, Taiwan・...",40K+ TWD / month,No requirement for relevant working experience,No management responsibility,Established in 1987 and headquartered in Taiwa...,https://www.cake.me/companies/tsmc/jobs/data-e...
1,數據工程師實習生DataEngineerIntern,COMMEET＿擁樂數據服務股份有限公司,1,Internship,Entry level,"Songshan District, Taipei City, Taiwan 105・Tai...",200 ~ 250 TWD / hour,No requirement for relevant working experience,No management responsibility,COMMEET (https://commeet.co) 擴大徵才，歡迎有熱情之新夥伴加入我...,https://www.cake.me/companies/commeet/jobs/dat...
2,資料工程師,上洋股份有限公司,1,Full-time,Assistant,"Wugu District, New Taipei, Taiwan",50K+ TWD / month,2 years of experience required,No management responsibility,數據驅動決策與營運最佳化是上洋邁向智慧營運的重要一步。我們正在尋找一位對資料工程具備熱情與專...,https://www.cake.me/companies/upyoung/jobs/dat...
3,[Bosch R&D - Internship]DataEngineerIntern,Bosch Vietnam,5,Full-time,Internship,"Ho Chi Minh City, Vietnam",No requirement for relevant working experience,No management responsibility,None,Company Description\nBosch first started in Vi...,https://www.cake.me/companies/bosch-vietnam-bd...
4,【國泰集團】數據工程師DataEngineer,國泰健康管理顧問股份有限公司,1,Full-time,Entry level,"Daan District, Taipei City, Taiwan",800K ~ 1M TWD / year,3 years of experience required,No management responsibility,【About Us】\n歡迎加入「國泰健康管理」！我們是一個充滿活力、追求創新、以數據和科技...,https://www.cake.me/companies/cathay-hcm/jobs/...
